In [1]:
%pip install -r ~/code/GitHub/UPennContrast/devops/girder/annotation_client/requirements.txt
%pip install -e ~/code/GitHub/UPennContrast/devops/girder/annotation_client/
%pip install pandas
%pip install imageio
%pip install rtree
%pip install scikit-image
%pip install shapely
%pip install nd2reader
%pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/arjunraj/code/GitHub/UPennContrast/devops/girder/annotation_client
  Attempting uninstall: annotation-client
    Found existing installation: annotation-client 0.1
    Uninstalling annotation-client-0.1:
      Successfully uninstalled annotation-client-0.1
  Running setup.py develop for annotation-client
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading 

In [11]:
import girder_client
import annotation_client.annotations as annotations
import annotation_client.tiles as tiles
import annotation_client.workers as workers
import pandas as pd
import annotation_tools
#import imageio
from shapely.geometry import Point, Polygon, LineString
#pd.set_option('display.max_colwidth', None)
import numpy as np

In [3]:
apiUrl = 'http://localhost:8080/api/v1'
username = 'admin'
password = 'password'

In [4]:
client = girder_client.GirderClient(apiUrl=apiUrl)
user_id = client.authenticate(username=username, password=password)['_id']
annotationClient = annotations.UPennContrastAnnotationClient(apiUrl=apiUrl, token=client.token)

In [5]:
datasets = dict()
for folder in annotationClient.client.listFolder(user_id,'user'):
    for dataset in annotationClient.client.listFolder(folder['_id']):
        datasets[dataset['name']] = dataset['_id']

pd.DataFrame(datasets.keys(),columns=['Dataset Name'])

,Dataset Name
0,111111_20210723_155544_798__Plate20210723_STR_...
1,111_20200110_2Dent_001
2,20200110_2Dent_001_maxmerge
3,"20221122_143426_960__WellA4_ChannelDAPI,CY3,A5..."
4,"20230314_175616_594__WellB2_ChannelDAPI,YFP,CY..."
...,...
173,Well2_50framesCOMPOSITENORECODE
174,Well2_50framesCOMPOSITENORECODE_maxmerge
175,Well_test_VID1630
176,"WellA1_ChannelBrightfield,YFP,CY5,DAPI,A594_Se..."


In [7]:
#dataset_name = '300cell-gastruloids_E11_1_04d16h00m'
dataset_name = 'CONNECT_TIME_normmedia_8well_col2_livecellgfp'

In [8]:
datasetId = datasets[dataset_name]
datasetClient = tiles.UPennContrastDataset(apiUrl=apiUrl, token=client.token, datasetId=datasetId)
print(datasetClient.tiles)
tile_width = datasetClient.tiles['tileWidth']
tile_height = datasetClient.tiles['tileHeight']
print(tile_width)
print(tile_height)

{'IndexRange': {'IndexC': 2, 'IndexT': 4, 'IndexXY': 6, 'IndexZ': 7}, 'IndexStride': {'IndexC': 1, 'IndexT': 14, 'IndexXY': 56, 'IndexZ': 2}, 'bandCount': 1, 'channelmap': {'Brightfield': 0, 'YFP': 1}, 'channels': ['Brightfield', 'YFP'], 'dtype': 'uint16', 'frames': [{'Channel': 'Brightfield', 'Frame': 0, 'Index': 0, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 0}, {'Channel': 'YFP', 'Frame': 1, 'Index': 0, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 0}, {'Channel': 'Brightfield', 'Frame': 2, 'Index': 1, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 1}, {'Channel': 'YFP', 'Frame': 3, 'Index': 1, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 1}, {'Channel': 'Brightfield', 'Frame': 4, 'Index': 2, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 2}, {'Channel': 'YFP', 'Frame': 5, 'Index': 2, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 2}, {'Channel': 'Brightfield', 'Frame': 6, 'Index': 3, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 3}, {'Channel': 'YFP', 'F

In [9]:
# May need to change the limit for large numbers of annotations. Default is 50.
pointAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, limit = 1000000, shape='point')
blobAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, limit = 1000000, shape='polygon')
lineAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, limit = 1000000, shape='line')
#allAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, limit = 1000000)
allAnnotationList = pointAnnotationList + blobAnnotationList + lineAnnotationList

In [10]:
print(len(blobAnnotationList))
print(len(pointAnnotationList))
print(len(allAnnotationList))
#print(len(lineAnnotationList))
print(blobAnnotationList[0])
print(pointAnnotationList[0])
#print(lineAnnotationList[0])

21
4
25
{'_id': '6504c0054bef1078719faf75', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 209, 'y': 614}, {'x': 206, 'y': 624}, {'x': 202, 'y': 628}, {'x': 194, 'y': 628}, {'x': 181, 'y': 622}, {'x': 180, 'y': 616}, {'x': 184, 'y': 608}, {'x': 193, 'y': 607}], 'datasetId': '6504bf104bef1078719faf21', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'polygon', 'tags': ['nucleus']}
{'_id': '6504c2224bef1078719fb525', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 431.76719305998256, 'y': 709.9848550087327, 'z': 0}], 'datasetId': '6504bf104bef1078719faf21', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'point', 'tags': ['YFP point']}


In [14]:
workerClient = workers.UPennContrastWorkerClient(datasetId, apiUrl=apiUrl, token=client.token, params = {'name':'test','id':'a878378','propertyType':'layer'})

annotationList = blobAnnotationList

annotation = annotationList[0]
print(annotation)

channel = annotation['channel']
location = annotation['location']
time, z, xy = location['Time'], location['Z'], location['XY']
frame = datasetClient.coordinatesToFrameIndex(xy, z, time, channel)
image = datasetClient.getRegion(datasetId, frame=frame)
print(image.shape)

#image = workerClient.get_image_for_annotation(annotation)




{'_id': '6504c0054bef1078719faf75', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 209, 'y': 614}, {'x': 206, 'y': 624}, {'x': 202, 'y': 628}, {'x': 194, 'y': 628}, {'x': 181, 'y': 622}, {'x': 180, 'y': 616}, {'x': 184, 'y': 608}, {'x': 193, 'y': 607}], 'datasetId': '6504bf104bef1078719faf21', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'polygon', 'tags': ['nucleus']}
(1022, 1024, 1)


In [22]:
from collections import defaultdict

channel = annotationList[0]['channel']

# 1. Group annotations by location using defaultdict
grouped_annotations = defaultdict(list)
for annotation in annotationList:
    location_key = (annotation['location']['Time'], annotation['location']['Z'], annotation['location']['XY'])
    grouped_annotations[location_key].append(annotation)

In [21]:
for location_key, annotations in grouped_annotations.items():
    time, z, xy = location_key
    frame = datasetClient.coordinatesToFrameIndex(xy, z, time, channel)
    image = datasetClient.getRegion(datasetId, frame=frame)

    if image is None:
        continue

    # 3. Compute properties for all annotations at that location
    for annotation in annotations:
        polygon = np.array([list(coordinate.values())[1::-1] for coordinate in annotation['coordinates']])
        mask = draw.polygon2mask(image.shape, polygon)
        intensities = image[mask]

        # Calculating the desired metrics
        mean_intensity = np.mean(intensities)
        max_intensity = np.max(intensities)
        min_intensity = np.min(intensities)
        median_intensity = np.median(intensities)
        q25_intensity = np.percentile(intensities, 25)
        q75_intensity = np.percentile(intensities, 75)
        total_intensity = np.sum(intensities)

        prop = {
            'MeanIntensity': float(mean_intensity),
            'MaxIntensity': float(max_intensity),
            'MinIntensity': float(min_intensity),
            'MedianIntensity': float(median_intensity),
            '25thPercentileIntensity': float(q25_intensity),
            '75thPercentileIntensity': float(q75_intensity),
            'TotalIntensity': float(total_intensity),
        }
        print(prop['MeanIntensity'])


dict_items([((0, 0, 0), [{'_id': '6504c0054bef1078719faf75', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 209, 'y': 614}, {'x': 206, 'y': 624}, {'x': 202, 'y': 628}, {'x': 194, 'y': 628}, {'x': 181, 'y': 622}, {'x': 180, 'y': 616}, {'x': 184, 'y': 608}, {'x': 193, 'y': 607}], 'datasetId': '6504bf104bef1078719faf21', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'polygon', 'tags': ['nucleus']}, {'_id': '6504c0e84bef1078719fb107', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 424, 'y': 325}, {'x': 422, 'y': 331}, {'x': 418, 'y': 335}, {'x': 406, 'y': 335}, {'x': 401, 'y': 333}, {'x': 399, 'y': 326}, {'x': 402, 'y': 321}, {'x': 410, 'y': 314}, {'x': 414, 'y': 310}, {'x': 421, 'y': 313}, {'x': 423, 'y': 318}], 'datasetId': '6504bf104bef1078719faf21', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'polygon',